<a href="https://colab.research.google.com/github/myeze/MachineLearningModels/blob/main/GPT2Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2 Model (Early Development)

**This notebook contains a model made by Myles Ezeanii.**

I am developing a Natural Language Processing (NLP) model based on the GPT-2 architecture in order to provide capabilities such as text generation, summarization, and conversational AI to users.

This project uses a pretrained GPT-2 model and dataset available from Hugging Face, and the implementation will utilize the PyTorch library.

---

For my dataset, I chose to use a premade, large-scale collection of data from Wikipedia articles. I felt this was best as Wikipedia provides an extensive and diverse repository of information, covering a wide range of topics and domains. This makes it well-suited for applications requiring broad contextual knowledge.

---

I established that my autotokenizer applies a Byte-Pair Encoding algorithm as well as parallel processing to the dataset in order to properly train the model.


---

My overall goal is for the model is to get a hands on experience of how to create a GPT Model and potentially create a large scale one using data from medical records in order to reccomend health resources/practices.


In [1]:
!pip install transformers>=4.28.0 # Update transformers to a more recent version that supports compute_metrics in TrainingArguments # Import tools for our Generative Pre-Trained Transformer
!pip install datasets==2.14.5 # Import text dataset to train model
!pip install torch==2.0.1 # Import PyTorch for testing

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.6.0 which is incompatible.
   ━━━━━━━━━━━━

In [2]:
from datasets import load_dataset, list_datasets # Import data set loader and a list of datasets we can choose from

ourDataset = load_dataset('wikitext', 'wikitext-2-raw-v1') # Gather large scale dataset from Wikepedia articles
#list_datasets() # Shows all the datasets we can choose from

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [3]:
# Automatically choose the right tokenizer based on our models name
from transformers import AutoTokenizer

# Load a pretrained gpt2 model (I chose 2 based on computational limitations)
ourTokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

# Define padding token to make shorter sequences the same length as the longest sequence in the batch of data
ourTokenizer.pad_token = ourTokenizer.eos_token # Use eos_token as pad_token

# Take a batch of examples as an input
def tokenizeFunction(examples):
    # Add padding and truncation for consistent sequence lengths
    return ourTokenizer(examples["text"], padding='max_length', truncation=True, max_length=128) # Break down the text of each example into smaller units (tokens) and return the output

# Apply the tokenize function to the dataset, process the data in batches, use parallel processing to make tokenization faster, and remove the original "text" column from the dataset
ourTokenizedDatasets = ourDataset.map(tokenizeFunction, batched=True, num_proc=4, remove_columns=["text"])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
print(ourTokenizer.__class__.__name__)  # Our Tokenizer uses a Byte-Pair Encoding algorithm
print(ourTokenizer.vocab_size)

GPT2TokenizerFast
50257


In [7]:
!pip uninstall torchvision -y
!pip install torchvision==0.15.2 --no-cache-dir

Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2-medium" ) # Choose a pre-trained GPT-2 model

# Function to add labels to the dataset
def add_labels(example):
    example['labels'] = example['input_ids'].copy()
    return example

ourTokenizedDatasets = ourTokenizedDatasets.map(add_labels)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    prediction_loss_only = True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ourTokenizedDatasets["train"],
    eval_dataset=ourTokenizedDatasets["validation"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print(results)

In [ ]:
import torch

    # Choose a prompt
    prompt = "The meaning of life is"

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(trainer.model.device)

    # Generate text
    generated_ids = trainer.model.generate(input_ids, max_length=50, num_return_sequences=1)

    # Decode the generated tokens
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Print the generated text
    print(generated_text)